### Imports

In [1]:
%load_ext autoreload
%autoreload 2

#### Independent Imports

In [2]:
import random
from collections import OrderedDict
import sys
import os
import cv2
import skimage.io
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

#### Location Dependent Imports

In [3]:
local = Path.home().group() == 'ademyanchuk'

In [4]:
if local:
    from chestxray.config import PANDA_PATH, PANDA_IMGS
    
    TEST_PATH = PANDA_IMGS
    ADD_DIR = Path("/home/ademyanchuk/Projects/ChestXRay/submit_panda")

In [5]:
if not local:
    ADD_DIR = Path("../input/panda0")
    EFFNET_DIR = "../input/TODO"
    sys.path.append(EFFNET_DIR)

In [6]:
sys.path.append(str(ADD_DIR))

In [25]:
# Imports from dataset
from submit_patch import PatchModel, TilesModel, aggregate
from submit_patch import CFG
from submit_patch import PatchTestDataset, TilesTestDataset

In [8]:
from efficientnet_pytorch import EfficientNet

In [9]:
if not local:
    from submit_patch import TEST_PATH, PANDA_PATH

### Setup

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
def seed_torch(seed=1982):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch()

In [12]:
os.listdir(PANDA_PATH)

['train_label_masks',
 'sample_submission.csv',
 'folds_dedup.csv',
 'test.csv',
 'train_images',
 'folds.csv',
 'folds_db.csv',
 'folds_cleaned.csv',
 'jpeg',
 'PANDA_Suspicious_Slides.csv',
 'hashes.npy',
 'hdf5',
 'train.csv']

In [13]:
TEST_PATH.exists()

True

### Data Loading

In [14]:
train = pd.read_csv(PANDA_PATH/'train.csv')
test = pd.read_csv(PANDA_PATH/'test.csv')
sample = pd.read_csv(PANDA_PATH/'sample_submission.csv')

In [15]:
train.head()

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


In [16]:
test.head()

,image_id,data_provider
0,005700be7e06878e6605e7a5a39de1b2,radboud
1,005c6e8877caf724c600fdce5d417d40,karolinska
2,0104f76634ff89bfff1ef0804a95c380,radboud


In [17]:
sample.head()

,image_id,isup_grade
0,005700be7e06878e6605e7a5a39de1b2,0
1,005c6e8877caf724c600fdce5d417d40,0
2,0104f76634ff89bfff1ef0804a95c380,0


### Classes Definitions

In [18]:
class PatchEnetModel(nn.Module):
    def __init__(self, backbone="efficientnet-b0", n=CFG.target_size, pretrained=True):
        super().__init__()
        assert backbone in ["efficientnet-b0"]

        # if we use BCE loss, need n-1 outputs
        if CFG.loss == "bce":
            n -= 1

        if pretrained:
            self.model = EfficientNet.from_pretrained(backbone)
        else:
            self.model = EfficientNet.from_name(backbone)

        num_ftrs = self.model._fc.in_features
        if CFG.model_cls == "deep":
            self.model._fc = nn.Sequential(
                OrderedDict(
                    [
                        (
                            "cls_fc",
                            nn.Linear(2 * num_ftrs, 512),
                        ),  # agregate use concat pooling, so *2
                        ("cls_bn", nn.BatchNorm1d(512)),
                        ("cls_relu", nn.ReLU(inplace=True)),
                        ("cls_logit", nn.Linear(512, n)),
                    ]
                )
            )
        elif CFG.model_cls == "one_layer":
            self.model._fc = nn.Sequential(
                OrderedDict([("cls_logit", nn.Linear(2 * num_ftrs, n))])
            )
        del self.model._avg_pooling  # use pooling in aggregate func

    def forward(self, x):
        batch_size, num_patch, C, H, W = x.shape

        x = x.view(-1, C, H, W)  # x -> bs*num_patch x C x H x W
        x = self.model.extract_features(
            x
        )  # x -> bs*num_patch x C(Maps) x H(Maps) x W(Maps)

        x = aggregate(x, batch_size, num_patch)
        x = self.model._dropout(x)
        x = self.model._fc(x)
        return x

### Model

In [20]:
ls {ADD_DIR}/22-06-2020-21-27_50_loss.pth

/home/ademyanchuk/Projects/ChestXRay/submit_panda/22-06-2020-21-27_50_loss.pth


In [21]:
def init_model(arch=CFG.arch, pretrained=False):
    if CFG.dataset == "patch":
        if arch == "efnet":
            model_ft = PatchEnetModel(pretrained=pretrained)
        else:
            model_ft = PatchModel(arch=CFG.arch, pretrained=pretrained)
    elif CFG.dataset == "tiles":
        if arch in ["resnet50", "resnet34"]:
            model_ft = TilesModel(arch=arch, pretrained=pretrained)
        else:
            print(f"Tiles model for {arch} is not yet implemented!")
    return model_ft

In [22]:
model_fnames = [
    "22-06-2020-21-27_50_loss.pth",
]
# Load Model/s
models = []
for name in model_fnames:
    model = init_model()

    weights_path = ADD_DIR/name
    checkpoint = torch.load(weights_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    models.append(model)
    
len(models)

1

### Inference

In [23]:
def compute_preds(outputs, loss_type=CFG.loss):
        if loss_type == "bce":
            preds = outputs.sigmoid().sum(1).data.cpu().round().numpy()
        else:
            preds = outputs.data.to("cpu").numpy().argmax(1)
        return preds.astype(int)

In [24]:
TARGET_SZ = CFG.target_size

if CFG.loss == "bce":
    TARGET_SZ = CFG.target_size - 1

def inference(models, test_loader, device):
    
    for model in models:
        model.to(device)
        model.eval()
    
    preds = []

    for i, images in enumerate(test_loader):    
        if CFG.aux_tile:
            img_main, img_aux = images
            img_main, img_aux = img_main.to(device), img_aux.to(device)
            y_preds = torch.zeros((len(img_main), TARGET_SZ), device=device)
        else:
            images = images.to(device)
            y_preds = torch.zeros((len(images), TARGET_SZ), device=device)
            
        with torch.no_grad():
            for model in models:
                if CFG.aux_tile:
                    p_main = model(img_main)
                    p_aux = model(img_aux)
                    y_preds += (p_main + p_aux) / 2
                else:
                    y_preds += model(images)
            y_preds = y_preds / len(models)
            
        pred = compute_preds(y_preds)
        preds.append(pred)

    preds = np.concatenate(preds)
    
    return preds

In [26]:
def submit(sample):
    if TEST_PATH.exists():
        if CFG.dataset == "patch":
            test_dataset = PatchTestDataset(sample, img_path=TEST_PATH)
        elif CFG.dataset == "tiles":
            test_dataset = TilesTestDataset(sample, img_path=TEST_PATH, aux_tile=CFG.aux_tile)
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)
        preds = inference(models, test_loader, device)
        sample['isup_grade'] = preds
    return sample

In [27]:
if local:
    sample = train.sample(50)
submission = submit(sample)
submission['isup_grade'] = submission['isup_grade'].astype(int)
submission.head()

,image_id,data_provider,isup_grade,gleason_score
5515,87c639ec5365c2d6d61581be6407c8b6,radboud,5,4+5
7977,c10eb86a927f359f5d1e573927f19945,radboud,1,3+3
9742,ec11cdbfe083f766ebc6a851353a1662,karolinska,1,3+4
611,0f83710916fcc2edf1a7a9a103fd48ac,radboud,2,3+4
1837,2e53a31a0727921e823b781914782036,karolinska,1,3+3


In [ ]:
permute = False
if TEST_PATH.exists() and permute:
    hard_cls_values = submission.loc[submission.isup_grade.isin([2,3,4]), "isup_grade"].copy().values
    np.random.shuffle(hard_cls_values)
    submission.loc[submission.isup_grade.isin([2,3,4]), "isup_grade"] = hard_cls_values
else:
    print("No hard class permutation")

In [ ]:
submission.to_csv('submission.csv', index=False)